[View in Colaboratory](https://colab.research.google.com/github/venabled/machine-learning-tutorial/blob/master/Convolutional_Neural_Network.ipynb)

In [0]:
from keras.datasets import cifar10
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, BatchNormalization, Conv2D
from keras.layers import MaxPool2D, GlobalAveragePooling2D
from keras.losses import categorical_crossentropy
from keras.regularizers import l2
from keras.callbacks import ReduceLROnPlateau
import keras.utils

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
x_train -= 128.0
x_test -= 128.0
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

170500096/170498071 [==============================] - 33s 0us/step


In [14]:
model = Sequential()

# Each of the convolutional blocks include a 3x3 kernel, relu, L2 regularization
# and a batch normalization layer
model.add(Conv2D(32, kernel_size=3, padding='same', input_shape=(32,32,3), kernel_regularizer=l2(0.002)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(32, kernel_size=3, padding='same', kernel_regularizer=l2(0.002)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(32, kernel_size=3, padding='same', kernel_regularizer=l2(0.002)))
model.add(Activation('relu'))
model.add(BatchNormalization())


# Add in some pooling to downsample by a factor of 2. After this downsample we
# will do more convolutions, but double the number of filters, keeping the total
# computation the same.
model.add(MaxPool2D())

model.add(Conv2D(64, kernel_size=3, padding='same', kernel_regularizer=l2(0.002)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(64, kernel_size=3, padding='same', kernel_regularizer=l2(0.002)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(64, kernel_size=3, padding='same', kernel_regularizer=l2(0.002)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(GlobalAveragePooling2D())
model.add(Dense(10))
model.add(Activation('softmax'))

# Actually build the model
model.compile(optimizer='nadam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_26 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_27 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_25 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_28 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_26 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 32, 32, 32)        9248      
__________

In [0]:
import keras.backend as K
K.get_value(model.optimizer.lr)

0.00020000001

In [13]:
history = model.fit(x_train, y_train,
                    epochs=25,
                    batch_size=64,
                    validation_data=(x_test, y_test),
                    verbose=1, 
                    callbacks=[ReduceLROnPlateau(patience=5)])

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 54s 1ms/step - loss: 1.4898 - acc: 0.5697 - val_loss: 1.4665 - val_acc: 0.5678
Epoch 2/100
50000/50000 [==============================] - 37s 744us/step - loss: 1.1292 - acc: 0.6895 - val_loss: 1.6592 - val_acc: 0.5562
Epoch 3/100
50000/50000 [==============================] - 37s 747us/step - loss: 1.0449 - acc: 0.7286 - val_loss: 1.1844 - val_acc: 0.6793
Epoch 4/100
50000/50000 [==============================] - 37s 747us/step - loss: 0.9990 - acc: 0.7491 - val_loss: 1.3731 - val_acc: 0.6460
Epoch 5/100
50000/50000 [==============================] - 37s 747us/step - loss: 0.9692 - acc: 0.7612 - val_loss: 1.5221 - val_acc: 0.6051
Epoch 6/100
50000/50000 [==============================] - 37s 741us/step - loss: 0.9501 - acc: 0.7687 - val_loss: 2.4149 - val_acc: 0.4977
Epoch 7/100
50000/50000 [==============================] - 37s 744us/step - loss: 0.9285 - acc: 0.7789 - val_los

KeyboardInterrupt: ignored

In [0]:
# lr = K.get_value(model.optimizer.lr)
# K.set_value(model.optimizer.lr, lr*0.60)
# print((lr, K.get_value(model.optimizer.lr)))

(2.0000001e-19, 1.2000001e-19)


In [0]:
plt.plot(history.epoch, history.history['loss'], history.epoch, history.history['val_loss'])
plt.legend(['Training Loss', 'Validation Loss'])